In [1]:
import numpy as np
import pandas as pd
import datetime

from numpy import linalg as la

from pandas_datareader import data

from sklearn.preprocessing import MinMaxScaler  
from sklearn.metrics import mean_squared_error

from keras import Sequential, layers, optimizers, losses

from matplotlib import pyplot as plt

Using TensorFlow backend.


### Load Data
We will use the following dataset:
https://data.world/perceptron/monthly-car-sales-quebec-1960
about monthly car sales in quebec from 1960. 
To download the dataset you have to join data.world. If you don't want to, just get it from my github repo.